<a href="https://colab.research.google.com/github/enigmaticcyph/ADOBE-INTER_IIT-12.0/blob/main/T_1_Glove_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LETS GET SHIT DONE

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
df=pd.read_excel("/content/drive/MyDrive/ADOBE INTER IIT/behaviour_simulation_train.xlsx",index_col=None,parse_dates=['date'])
df.head()

,id,date,likes,content,username,media,inferred company
0,1,2020-12-12 00:47:00,1,"Spend your weekend morning with a Ham, Egg, an...",TimHortonsPH,[Photo(previewUrl='https://pbs.twimg.com/media...,tim hortons
1,2,2018-06-30 10:04:20,2750,Watch rapper <mention> freestyle for over an H...,IndyMusic,[Photo(previewUrl='https://pbs.twimg.com/media...,independent
2,3,2020-09-29 19:47:28,57,Canadian Armenian community demands ban on mil...,CBCCanada,[Photo(previewUrl='https://pbs.twimg.com/media...,cbc
3,4,2020-10-01 11:40:09,152,"1st in Europe to be devastated by COVID-19, It...",MKWilliamsRome,[Photo(previewUrl='https://pbs.twimg.com/media...,williams
4,5,2018-10-19 14:30:46,41,Congratulations to Pauletha Butts of <mention>...,BGISD,[Photo(previewUrl='https://pbs.twimg.com/media...,independent


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                300000 non-null  int64         
 1   date              300000 non-null  datetime64[ns]
 2   likes             300000 non-null  int64         
 3   content           300000 non-null  object        
 4   username          300000 non-null  object        
 5   media             300000 non-null  object        
 6   inferred company  300000 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 16.0+ MB


In [ ]:
print(df.isnull().sum())
print(df.shape)

id                  0
date                0
likes               0
content             0
username            0
media               0
inferred company    0
dtype: int64
(300000, 7)


# Feature Engineering :

In [ ]:
def classify_content(link):
  pattern='\[([a-z A-z]*)\('
  l=re.findall(pattern, link)
  if 'Video' in l :
    return 'Video'
  elif 'Photo' in l:
    return 'Photo'
  elif 'Gif' in l:
    return 'Gif'
  else:
    return "UNKOWN"

In [ ]:
def extract_url(text):
  url=re.findall(r'\'(https\:\/\/[a-zA-Z\.\/\_0-9\-\?\=\&\#\>\<\$\%\@]*)\'', text)## Maa ki chut chatGPT ki ## fuck Elon
  return url[0]

In [ ]:
stop_words=set(stopwords.words('english'))
stop_words.update(['hyperlink','st','nd','rd','th'])
# print(type(stop_words))
lemmatizer=WordNetLemmatizer()
def pre_processing(text):
  text = re.sub(r'[^a-zA-Z\s]', ' ', text)
  text=text.lower()
  tokens=word_tokenize(text)
  cleaned_tokens=[lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
  return ' '.join(cleaned_tokens)

In [ ]:
df['content_type'] = df['media'].apply(classify_content)
df['hour_of_day']=df['date'].dt.hour
df['day_of_week']=df['date'].dt.dayofweek
df['day_of_month']=df['date'].dt.day
df['month']=df['date'].dt.month
df['Photo/thumbnail']=df['media'].apply(extract_url)
df['clean_content']=df['content'].apply(pre_processing)
df.head()

,id,date,likes,content,username,media,inferred company,content_type,hour_of_day,day_of_week,day_of_month,month,Photo/thumbnail,clean_content
0,1,2020-12-12 00:47:00,1,"Spend your weekend morning with a Ham, Egg, an...",TimHortonsPH,[Photo(previewUrl='https://pbs.twimg.com/media...,tim hortons,Photo,0,5,12,12,https://pbs.twimg.com/media/Eo8N3JLVoAAlDJT?fo...,spend weekend morning ham egg cheese wrap pair...
1,2,2018-06-30 10:04:20,2750,Watch rapper <mention> freestyle for over an H...,IndyMusic,[Photo(previewUrl='https://pbs.twimg.com/media...,independent,Photo,10,5,30,6,https://pbs.twimg.com/media/Dg7mW-VX0AE_hMn?fo...,watch rapper mention freestyle hour
2,3,2020-09-29 19:47:28,57,Canadian Armenian community demands ban on mil...,CBCCanada,[Photo(previewUrl='https://pbs.twimg.com/media...,cbc,Photo,19,1,29,9,https://pbs.twimg.com/media/EjG2s4aXgAMNM1o?fo...,canadian armenian community demand ban militar...
3,4,2020-10-01 11:40:09,152,"1st in Europe to be devastated by COVID-19, It...",MKWilliamsRome,[Photo(previewUrl='https://pbs.twimg.com/media...,williams,Photo,11,3,1,10,https://pbs.twimg.com/media/EjPaVniX0AAaWLJ?fo...,europe devastated covid italy redoubled effort...
4,5,2018-10-19 14:30:46,41,Congratulations to Pauletha Butts of <mention>...,BGISD,[Photo(previewUrl='https://pbs.twimg.com/media...,independent,Photo,14,4,19,10,https://pbs.twimg.com/media/Dp4L0cSUcAAh9JG?fo...,congratulation pauletha butt mention presented...


In [ ]:
len(df['Photo/thumbnail'].unique())

295145

In [ ]:
df_text=df[['username','inferred company','content_type','hour_of_day','day_of_week','day_of_month','month','clean_content','likes']]
df_image=df[['clean_content','content_type','Photo/thumbnail','likes']]

In [ ]:
df_text.head()

,username,inferred company,content_type,hour_of_day,day_of_week,day_of_month,month,clean_content,likes
0,TimHortonsPH,tim hortons,Photo,0,5,12,12,spend weekend morning ham egg cheese wrap pair...,1
1,IndyMusic,independent,Photo,10,5,30,6,watch rapper mention freestyle hour,2750
2,CBCCanada,cbc,Photo,19,1,29,9,canadian armenian community demand ban militar...,57
3,MKWilliamsRome,williams,Photo,11,3,1,10,europe devastated covid italy redoubled effort...,152
4,BGISD,independent,Photo,14,4,19,10,congratulation pauletha butt mention presented...,41


In [ ]:
def load_glove_embeddings(glove_file):
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

# Load GloVe vectors (update the path to your GloVe file)
glove_embeddings = load_glove_embeddings('/content/drive/MyDrive/ADOBE INTER IIT/glove.6B.50d.txt')

# Check the size of the embeddings
print(f'Loaded {len(glove_embeddings)} word vectors.')


Loaded 400001 word vectors.


In [ ]:
def vectorize_text(tokens):
    if len(tokens) == 0:
        return np.zeros(50)  # Assuming 100-dimensional GloVe vectors
    word_vectors = [glove_embeddings[word] for word in tokens if word in glove_embeddings]
    return np.mean(word_vectors, axis=0)  # Average the word vectors

# Vectorize the processed captions
df_text['features'] = df_text['clean_content'].apply(vectorize_text)

# Convert the features column to a suitable format
X_text = np.array(df_text['features'].tolist())


<ipython-input-13-c7d0b9b3216e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text['features'] = df_text['clean_content'].apply(vectorize_text)


In [ ]:
le=LabelEncoder()
for i in ['username',	'inferred company','content_type'	]:
  df_text[i]=le.fit_transform(df_text[i])


<ipython-input-14-2749c5f59018>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text[i]=le.fit_transform(df_text[i])
<ipython-input-14-2749c5f59018>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_text[i]=le.fit_transform(df_text[i])
<ipython-input-14-2749c5f59018>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

In [ ]:
df_text.head()

,username,inferred company,content_type,hour_of_day,day_of_week,day_of_month,month,clean_content,likes,features
0,1572,190,1,0,5,12,12,spend weekend morning ham egg cheese wrap pair...,1,"[-0.024347508, 0.83851093, 0.7278812, 0.702687..."
1,775,98,1,10,5,30,6,watch rapper mention freestyle hour,2750,"[-0.11757295, 0.8646633, 0.80544746, 0.8170384..."
2,260,37,1,19,1,29,9,canadian armenian community demand ban militar...,57,"[-0.061728932, 0.8281797, 0.63011825, 0.654013..."
3,913,214,1,11,3,1,10,europe devastated covid italy redoubled effort...,152,"[-0.011977817, 0.7912272, 0.604498, 0.7325648,..."
4,187,98,1,14,4,19,10,congratulation pauletha butt mention presented...,41,"[-0.08245936, 0.80912906, 0.72853386, 0.733436..."


In [ ]:
X_additional= df_text[['username','inferred company','content_type','hour_of_day', 'day_of_week', 'day_of_month', 'month']].values
X_combined = np.hstack([X_additional, X_text])
y=df['likes'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K

# Define custom R² metric using TensorFlow backend
def r2_metric(y_true, y_pred):
    # Ensure y_true is casted to the same type as y_pred
    y_true = K.cast(y_true, y_pred.dtype)
    ss_res = K.sum(K.square(y_true - y_pred))  # Sum of squared residuals
    ss_tot = K.sum(K.square(y_true - K.mean(y_true)))  # Total sum of squares
    return 1 - ss_res / (ss_tot + K.epsilon())  # R² score formula


In [ ]:
# Create the neural network model
model = Sequential()
model.add(Dense(128, input_dim=X_combined.shape[1], activation='relu'))  # Input layer
# model.add(Dropout(0.2))  # Dropout for regularization
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
# model.add(Dropout(0.2))  # Another dropout layer
model.add(Dense(1, activation='linear'))  # Output layer (for regression)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Set up early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32, callbacks=[early_stopping])


Epoch 1/50
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - loss: 26586194.0000 - mse: 26586194.0000 - val_loss: 23126424.0000 - val_mse: 23126424.0000
Epoch 2/50
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - loss: 24979786.0000 - mse: 24979786.0000 - val_loss: 23146856.0000 - val_mse: 23146856.0000
Epoch 3/50
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 22423130.0000 - mse: 22423130.0000 - val_loss: 23013010.0000 - val_mse: 23013010.0000
Epoch 4/50
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - loss: 25865696.0000 - mse: 25865696.0000 - val_loss: 22914698.0000 - val_mse: 22914698.0000
Epoch 5/50
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - loss: 24012086.0000 - mse: 24012086.0000 - val_loss: 22921494.0000 - val_mse: 22921494.0000
Epoch 6/50
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 24243522.0000 - mse: 24243522.0000 - val_loss: 22813884.0000 - val_mse: 22813884.0000
Epoch 7/50
7500/7500 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step - loss: 22881326.0000 - mse: 22881326.0000 - val_lo

In [ ]:
# Evaluate the model on the test set
loss, mae = model.evaluate(X_test, y_test)
print(f'Mean Absolute Error: {mae}')


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 13999634.0000 - mse: 13999634.0000
Mean Absolute Error: 22835068.0


In [ ]:
from sklearn.metrics import r2_score
y_pred = model.predict(X_test)

# Calculate R² score
r2 = r2_score(y_test, y_pred)
print(f'R² Score: {r2}')

# Evaluate the model on the test set
loss, mae = model.evaluate(X_test, y_test)
print(f'Mean Absolute Error: {mae}')

1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
R² Score: 0.02080059051513672
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 13999634.0000 - mse: 13999634.0000
Mean Absolute Error: 22835068.0
